In [1]:
import numpy as np
import cPickle
import os
import gzip

# First exercise: Classifying MNIST with MLPs
In this exercise you will implement a Neural Network (or MLP) and classify the MNIST digits with it.
MNIST is a "well hung" dataset that has been used a lot over the years to benchmark different classification algorithms. 
To learn more about it have a look here: http://yann.lecun.com/exdb/mnist/ .

# Data Loading
We first define a function for downloading and loading MNIST.
**WARNING**: Executing it will obviously use up some space on your machine ;). 

In [2]:
def mnist(datasets_dir='./data'):
    if not os.path.exists(datasets_dir):
        os.mkdir(datasets_dir)
    data_file = os.path.join(datasets_dir, 'mnist.pkl.gz')
    if not os.path.exists(data_file):
        print('... downloading MNIST from the web')
        try:
            import urllib
            urllib.urlretrieve('http://google.com')
        except AttributeError:
            import urllib.request as urllib
        url = 'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
        urllib.urlretrieve(url, data_file)
        

    print('... loading data')
    # Load the dataset
    f = gzip.open(data_file, 'rb')
    try:
        train_set, valid_set, test_set = cPickle.load(f, encoding="latin1")
    except TypeError:
        train_set, valid_set, test_set = cPickle.load(f)
    f.close()

    test_x, test_y = test_set
    test_x = test_x.astype('float32')
    test_x = test_x.astype('float32').reshape(test_x.shape[0], 1, 28, 28)
    test_y = test_y.astype('int32')
    valid_x, valid_y = valid_set
    valid_x = valid_x.astype('float32')
    valid_x = valid_x.astype('float32').reshape(valid_x.shape[0], 1, 28, 28)
    valid_y = valid_y.astype('int32')
    train_x, train_y = train_set
    train_x = train_x.astype('float32').reshape(train_x.shape[0], 1, 28, 28)
    train_y = train_y.astype('int32')
    rval = [(train_x, train_y), (valid_x, valid_y), (test_x, test_y)]
    print('... done loading data')
    return rval

# Neural Network Layers
We now define "bare bone" neural network layers.
The parts marked with **TODO** are where you should finish the implementation!
Conceptually we will implement the layers as follows:

Each layer has a constructor that takes an input layer plus some additional arguments such as layer size and the activation function name. The layer then uses the provided input layer to compute the layer dimensions, weight shapes, etc. and setup all auxilliary variables.

Each layer then has to provide three functions (as defined in the Layer class below): *output_shape()*, *fprop()* and *brop()*. The output_shape function is used to figure out the shape for the next layer and the *fprop()/bprop()* functions are used to compute forward and backward passes through the network.

In [3]:
# start by defining simple helpers
def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def sigmoid_d(x):
    return sigmoid(x)*(1 - sigmoid(x))

def tanh(x):
    return np.tanh(x)

def tanh_d(x):
    return 1.0 - np.tanh(x)**2

def relu(x):
    return np.maximum(0.0, x)

relu_d = np.vectorize(lambda x: 1.0 if x > 0 else 0.0)

def softmax(x, axis=1):
    # first subtracted the maximum to make this safe!
    x_max = np.max(x, axis=axis, keepdims=True)
    x_safe = x - x_max
    e_x = np.exp(x_safe)
    return e_x / np.sum(e_x, axis=axis, keepdims=True)

def one_hot(labels):
    """this creates a one hot encoding from a flat vector:
    i.e. given y = [0,2,1]
     it creates y_one_hot = [[1,0,0], [0,0,1], [0,1,0]]
    """
    classes = np.unique(labels)
    n_classes = classes.size
    one_hot_labels = np.zeros(labels.shape + (n_classes,))
    for c in classes:
        one_hot_labels[labels == c, c] = 1
    return one_hot_labels

def unhot(one_hot_labels):
    """ Invert a one hot encoding, creating a flat vector """
    return np.argmax(one_hot_labels, axis=-1)

# then define an activation function class
class Activation(object):
    
    def __init__(self, tname):
        if tname == 'sigmoid':
            self.act = sigmoid
            self.act_d = sigmoid_d
        elif tname == 'tanh':
            self.act = tanh
            self.act_d = tanh_d
        elif tname == 'relu':
            self.act = relu
            self.act_d = relu_d
        else:
            raise ValueError('Invalid activation function.')
            
    def fprop(self, input, dropout):
        self.last_input = self.dropout(input.copy()) if dropout else input.copy()
        return self.act(input)
    
    def dropout(self, input):
        # completely "freeze" nodes with a chance of 1/2 each
        for i in range(input.shape[0]):
            input[i] = np.zeros_like(input[i]) if random.random() < 0.5 else input[i]
        return input
    
    def bprop(self, output_grad):
        return output_grad * self.act_d(self.last_input)

# define a base class for layers
class Layer(object):
    
    def fprop(self, input, dropout):
        """ Calculate layer output for given input 
            (forward propagation). 
        """
        raise NotImplementedError('This is an interface class, please use a derived instance')

    def bprop(self, output_grad):
        """ Calculate input gradient and gradient 
            with respect to weights and bias (backpropagation). 
        """
        raise NotImplementedError('This is an interface class, please use a derived instance')

    def output_size(self):
        """ Calculate size of this layer's output.
        input_shape[0] is the number of samples in the input.
        input_shape[1:] is the shape of the feature.
        """
        raise NotImplementedError('This is an interface class, please use a derived instance')

# define a base class for loss outputs
# an output layer can then simply be derived
# from both Layer and Loss 
class Loss(object):

    def loss(self, output, output_net):
        """ Calculate mean loss given real output and network output. """
        raise NotImplementedError('This is an interface class, please use a derived instance')

    def input_grad(self, output, output_net):
        """ Calculate input gradient real output and network output. """
        raise NotImplementedError('This is an interface class, please use a derived instance')

# define a base class for parameterized things
class Parameterized(object):
    
    def params(self):
        """ Return parameters (by reference) """
        raise NotImplementedError('This is an interface class, please use a derived instance')
    
    def grad_params(self):
        """ Return accumulated gradient with respect to params. """
        raise NotImplementedError('This is an interface class, please use a derived instance')

# define a container for providing input to the network
class InputLayer(Layer):
    
    def __init__(self, input_shape):
        if not isinstance(input_shape, tuple):
            raise ValueError("InputLayer requires input_shape as a tuple")
        self.input_shape = input_shape

    def output_size(self):
        return self.input_shape
    
    def fprop(self, input, dropout):
        return input
    
    def bprop(self, output_grad):
        return output_grad
        
class FullyConnectedLayer(Layer, Parameterized):
    """ A standard fully connected hidden layer, as discussed in the lecture.
    """
    
    def __init__(self, input_layer, num_units, 
                 init_stddev, activation_fun=Activation('relu')):
        self.num_units = num_units
        self.activation_fun = activation_fun
        
        self.input_shape = input_layer.output_size()

        # initialize weight matrix with random noise
        self.W = np.random.normal(loc=0.0, scale=init_stddev, size=(self.input_shape[1], num_units))
        
        # initialize biases vector with random noise
        self.b = np.random.normal(loc=0.0, scale=init_stddev, size=(num_units,))

        self.dW = None
        self.db = None
    
    def output_size(self):
        return (self.input_shape[0], self.num_units)
    
    def fprop(self, input, dropout):
        self.last_input = input.copy()
        z = input.dot(self.W) + self.b
        
        return self.activation_fun.fprop(z, dropout) if self.activation_fun else z
        
    def bprop(self, output_grad):
        """ Calculate input gradient (backpropagation). """
        n = output_grad.shape[0]
        
        # gradient wrt. last layer activations
        dL_da = self.activation_fun.bprop(output_grad) if self.activation_fun else output_grad
        
        # weight matrix gradient
        self.dW = self.last_input.T.dot(dL_da)
        
        # biases vector gradient
        self.db = np.sum(dL_da, axis=0)
        
        return dL_da.dot(self.W.T)
    
    def params(self):
        return self.W, self.b

    def grad_params(self):
        return self.dW, self.db

# finally we specify the interface for output layers 
# which are layers that also have a loss function
# we will implement two output layers:
#  a Linear, and Softmax (Logistic Regression) layer
# The difference between output layers and and normal 
# layers is that they will be called to compute the gradient
# of the loss through input_grad(). bprop will never 
# be called on them!
class LinearOutput(Layer, Loss):
    """ A simple linear output layer that  
        uses a squared loss (e.g. should be used for regression)
    """
    def __init__(self, input_layer):
        self.input_size = input_layer.output_size()
        
    def output_size(self):
        return (1,)
        
    def fprop(self, input, dropout):
        return input

    def bprop(self, output_grad):
        raise NotImplementedError(
            'LinearOutput should only be used as the last layer of a Network'
            + ' bprop() should thus never be called on it!'
        )
    
    def input_grad(self, Y, Y_pred):
        # divided by n also for gradient checking to work
        return (Y_pred - Y) / Y.shape[0]

    def loss(self, Y, Y_pred):
        # calculate mean squared loss
        loss = 0.5 * np.square(Y_pred - Y)
        return np.mean(np.sum(loss, axis=1))

class SoftmaxOutput(Layer, Loss):
    """ A softmax output layer that calculates 
        the negative log likelihood as loss
        and should be used for classification.
    """
    
    def __init__(self, input_layer):
        self.input_size = input_layer.output_size()
        
    def output_size(self):
        return (1,)
    
    def fprop(self, input, dropout):
        return softmax(input)
    
    def bprop(self, output_grad):
        raise NotImplementedError(
            'SoftmaxOutput should only be used as the last layer of a Network'
            + ' bprop() should thus never be called on it!'
        )
    
    def input_grad(self, Y, Y_pred):
        return (Y_pred - Y) / Y.shape[0]

    def loss(self, Y, Y_pred):
        eps = 1e-10
        loss = np.sum(-np.log(Y_pred+eps)*Y, axis=1)
        return np.mean(loss)
        

In [4]:

class NeuralNetwork:
    """ Our Neural Network container class.
    """
    def __init__(self, layers):
        self.layers = layers
        self.dWSum = [0 for _ in range(len(self.layers))]
        self.dbSum = [0 for _ in range(len(self.layers))]
        
    def _loss(self, X, Y):
        Y_pred = self.predict(X, dropout=False)
        return self.layers[-1].loss(Y, Y_pred)

    def predict(self, X, dropout):
        """ Calculate an output Y for the given input X. """
        current = X
        for layer in self.layers:
            current = layer.fprop(current, dropout)
        return current
    
    def backpropagate(self, Y, Y_pred, upto=0):
        """ Backpropagation of partial derivatives through 
            the complete network up to layer 'upto'
        """
        next_grad = self.layers[-1].input_grad(Y, Y_pred)
        for i in reversed(range(upto, len(self.layers)-1)):
            next_grad = self.layers[i].bprop(next_grad)
        return next_grad
    
    def classification_error(self, X, Y):
        """ Calculate error on the given data 
            assuming they are classes that should be predicted. 
        """
        Y_pred = unhot(self.predict(X, dropout=False))
        error = Y_pred != Y
        return np.mean(error)
    
    def update_weights(self, learning_rate, adagrad=False):
        """ Update the weights of all layers with parameters.
        """
        if adagrad:
            for i, layer in enumerate(self.layers):
                if isinstance(layer, Parameterized):
                    self.adagrad_update_sum(i)
                    lr_weights = self.adagrad_get_learning_rate_weights(learning_rate, i)
                    lr_bias = self.adagrad_get_learning_rate_bias(learning_rate, i)
                    layer.W -= np.multiply(lr_weights, layer.dW)
                    layer.b -= np.multiply(lr_bias, layer.db)
        else:
            for layer in self.layers:
                if isinstance(layer, Parameterized):
                    layer.W -= learning_rate * layer.dW
                    layer.b -= learning_rate * layer.db
                    
    
    def adagrad_update_sum(self, index):
        self.dWSum[index] += self.layers[index].dW**2
        self.dbSum[index] += self.layers[index].db**2
        
    
    def adagrad_get_learning_rate_weights(self, learning_rate, index, epsilon=1e-8):
        return learning_rate / np.sqrt(self.dWSum[index]+epsilon)
    

    def adagrad_get_learning_rate_bias(self, learning_rate, index, epsilon=1e-8):
        return learning_rate / np.sqrt(self.dbSum[index]+epsilon)
                
    
    def sgd_epoch(self, X, Y, learning_rate, batch_size, dropout, adagrad=False):
        n_samples = X.shape[0]
        n_batches = n_samples // batch_size
        for b in range(n_batches):
            if random.random() > 0.2:
                continue
            index_start = b * batch_size
            index_end = (b + 1) * batch_size
            prediction = self.predict(X[index_start:index_end], dropout)
            self.backpropagate(Y[index_start:index_end], prediction)
            self.update_weights(learning_rate, adagrad)
        # actually here there might be some samples missing because
        # batchsize might not divide cleanly. Tutor said this was okay
    
    def gd_epoch(self, X, Y, learning_rate, dropout, adagrad=False):
        prediction = self.predict(X, dropout)
        self.backpropagate(Y, prediction)
        self.update_weights(learning_rate, adagrad)
    
    def train(self, X, Y, X_val, Y_val, learning_rate=0.1, max_epochs=100, 
              batch_size=64, descent_type="sgd", y_one_hot=True, dropout=True, adagrad=False):
        """ Train network on the given data. """
        n_samples = X.shape[0]
        n_batches = n_samples // batch_size
        if y_one_hot:
            Y_train = one_hot(Y)
            Y_valOneHot = one_hot(Y_val)
        else:
            Y_train = Y
            Y_valOneHot = Y_val
        print("... starting training")
        for e in range(max_epochs+1):
            if descent_type == "sgd":
                self.sgd_epoch(X, Y_train, learning_rate, batch_size, dropout, adagrad)
            elif descent_type == "gd":
                self.gd_epoch(X, Y_train, learning_rate, dropout, adagrad)
            else:
                raise NotImplementedError("Unknown gradient descent type {}".format(descent_type))

            # Output error on the training data
            print("epoch: " + str(e))
            train_loss = self._loss(X, Y_train)
            train_error = self.classification_error(X, Y)
            validation_loss = self._loss(X_val, Y_valOneHot)
            validation_error = self.classification_error(X_val, Y_val)
            print('epoch {:.4f}, loss {:.4f}, train error {:.4f}, validation error {:.4f}'.format(e, train_loss, train_error, validation_error))
    
    def check_gradients(self, X, Y):
        """ Helper function to test the parameter gradients for
        correctness. """
        for l, layer in enumerate(self.layers):
            if isinstance(layer, Parameterized):
                print('checking gradient for layer {}'.format(l))
                for p, param in enumerate(layer.params()):
                    # we iterate through all parameters
                    param_shape = param.shape
                    # define functions for conveniently swapping
                    # out parameters of this specific layer and 
                    # computing loss and gradient with these 
                    # changed parametrs
                    def output_given_params(param_new):
                        """ A function that will compute the output 
                            of the network given a set of parameters
                        """
                        # copy provided parameters
                        param[:] = np.reshape(param_new, param_shape)
                        # return computed loss
                        return self._loss(X, Y)

                    def grad_given_params(param_new):
                        """A function that will compute the gradient 
                           of the network given a set of parameters
                        """
                        # copy provided parameters
                        param[:] = np.reshape(param_new, param_shape)
                        # Forward propagation through the net
                        Y_pred = self.predict(X, dropout=False)
                        # Backpropagation of partial derivatives
                        self.backpropagate(Y, Y_pred, upto=l)
                        # return the computed gradient 
                        return np.ravel(self.layers[l].grad_params()[p])

                    # let the initial parameters be the ones that
                    # are currently placed in the network and flatten them
                    # to a vector for convenient comparisons, printing etc.
                    param_init = np.ravel(np.copy(param))
                    
                    # TODO ####################################
                    # TODO compute the gradient with respect to
                    #      the initial parameters in two ways:
                    #      1) with grad_given_params()
                    #      2) with finite differences 
                    #         using output_given_params()
                    #         (as discussed in the lecture)
                    #      if your implementation is correct 
                    #      both results should be epsilon close
                    #      to each other!
                    # TODO ####################################
                    epsilon = 1e-4
                    # making sure your gradient checking routine itself 
                    # has no errors can be a bit tricky. To debug it
                    # you can "cheat" by using scipy which implements
                    # gradient checking exactly the way you should!
                    # To do that simply run the following here:
                    
                    loss_base = output_given_params(param_init)
                    # TODO this should hold the gradient as calculated through bprop
                    gparam_bprop = grad_given_params(param_init)
                    # TODO this should hold the gradient calculated through 
                    #      finite differences
                    gparam_fd = np.zeros_like(param_init)
                    # calculate difference between them
                    for i in range(len(param_init)):
                        if i == 0 or (i % 200 == 0):
                            print('calculate param_hat index {} of {}'.format(i, len(param_init)-1))
                        p = param_init.copy()
                        p[i] += epsilon
                        gparam_fd[i] = (output_given_params(p) - loss_base) / epsilon
                    err = np.mean(np.abs(gparam_bprop - gparam_fd))
                    print('diff {:.2e}'.format(err))
                    
                    assert(err < epsilon)
                    
                    # reset the parameters to their initial values
                    param[:] = np.reshape(param_init, param_shape)

# Neural Network class
With all layers in place (and properly implemented by you) we can finally define a neural network.
For our purposes a neural network is simply a collection of layers which we will cycle through and on which we will call fprop and bprop to compute partial derivatives with respect to the input and the parameters.

Pay special attention to the *check_gradients()* function in which you should implement automatic differentiation. This function will become your best friend when checking the correctness of your implementation.

# Gradient Checking
After implementing everything it is always a good idea to setup some layers and perform gradient
checking on random data. **Note** that this is only an example! It is not a useful network architecture ;). We also expect you to play around with this to test all your implemented components.

In [5]:
input_shape = (10, 10)
n_labels = 10
layers = [InputLayer(input_shape)]

layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=40,
                init_stddev=0.1,
                activation_fun=Activation('relu')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=20,
                init_stddev=0.1,
                activation_fun=Activation('tanh')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=n_labels,
                init_stddev=0.1,
                activation_fun=Activation('relu')
))
layers.append(SoftmaxOutput(layers[-1]))
nn = NeuralNetwork(layers)

In [6]:
# create random data
X = np.random.normal(size=input_shape)
# and random labels
Y = np.zeros((input_shape[0], n_labels))
for i in range(Y.shape[0]):
    idx = np.random.randint(n_labels)
    Y[i, idx] = 1.

In [7]:
nn.check_gradients(X, Y)

checking gradient for layer 1
calculate param_hat index 0 of 399
calculate param_hat index 200 of 399
diff 6.23e-07
calculate param_hat index 0 of 39
diff 1.86e-08
checking gradient for layer 2
calculate param_hat index 0 of 799
calculate param_hat index 200 of 799
calculate param_hat index 400 of 799
calculate param_hat index 600 of 799
diff 1.46e-08
calculate param_hat index 0 of 19
diff 2.12e-07
checking gradient for layer 3
calculate param_hat index 0 of 199
diff 4.55e-08
calculate param_hat index 0 of 9
diff 2.09e-06


# Training on MNIST
Finally we can let our network run on the MNIST dataset!

First load the data and reshape it.

In [8]:
# load
Dtrain, Dval, Dtest = mnist()
X_train, y_train = Dtrain
X_val, y_val = Dval
# Downsample training data to make it a bit faster for testing this code
n_train_samples = 50000
train_idxs = np.random.permutation(X_train.shape[0])[:n_train_samples]
X_train = X_train[train_idxs]
y_train = y_train[train_idxs]

... loading data
... done loading data


*Dtrain* contains 50k images which are of size 28 x 28 pixels. Hence:

In [9]:
print("X_train shape: {}".format(np.shape(X_train)))
print("y_train shape: {}".format(np.shape(y_train)))

X_train shape: (50000, 1, 28, 28)
y_train shape: (50000,)


y_train will automatically be converted in the *train()* function to one_hot encoding.


But we need to reshape X_train, as our Network expects flat vectors of size 28*28 as input!

In [10]:
X_train = X_train.reshape(X_train.shape[0], -1)
X_val = X_val.reshape(X_val.shape[0], -1)
print("Reshaped X_train size: {}".format(X_train.shape))

Reshaped X_train size: (50000, 784)


Ah, much better ;-)! 

Now we can finally really start training a Network!


I pre-defined a small Network for you below. Again This is not really a good default and will not produce state of the art results. Please play around with this a bit. See how different activation functions and training procedures (gd / sgd) affect the result.

In [11]:
import time
import random 

# Setup a small MLP / Neural Network
# we can set the first shape to None here to indicate that
# we will input a variable number inputs to the network
input_shape = (None, 28*28)
layers = [InputLayer(input_shape)]
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=170,
                init_stddev=0.01,
                activation_fun=Activation('sigmoid')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=100,
                init_stddev=0.01,
                activation_fun=Activation('relu')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=10,
                init_stddev=0.01,
                # last layer has no nonlinearity 
                # (softmax will be applied in the output layer)
                activation_fun=None 
))
layers.append(SoftmaxOutput(layers[-1]))

nn = NeuralNetwork(layers)
# Train neural network
t0 = time.time()
nn.train(X_train, y_train, X_val, y_val, learning_rate=0.1, 
         max_epochs=75, batch_size=64, y_one_hot=True, dropout=False, adagrad=True)
t1 = time.time()
print('Duration: {:.1f}s'.format(t1-t0))

... starting training
epoch: 0
epoch 0.0000, loss 0.8476, train error 0.2850, validation error 0.2711
epoch: 1
epoch 1.0000, loss 0.3955, train error 0.1139, validation error 0.1027
epoch: 2
epoch 2.0000, loss 0.3155, train error 0.0887, validation error 0.0823
epoch: 3
epoch 3.0000, loss 0.2829, train error 0.0817, validation error 0.0760
epoch: 4
epoch 4.0000, loss 0.2342, train error 0.0662, validation error 0.0608
epoch: 5
epoch 5.0000, loss 0.2132, train error 0.0615, validation error 0.0602
epoch: 6
epoch 6.0000, loss 0.1916, train error 0.0559, validation error 0.0549
epoch: 7
epoch 7.0000, loss 0.1807, train error 0.0529, validation error 0.0511
epoch: 8
epoch 8.0000, loss 0.1771, train error 0.0513, validation error 0.0534
epoch: 9
epoch 9.0000, loss 0.1668, train error 0.0501, validation error 0.0512
epoch: 10
epoch 10.0000, loss 0.1543, train error 0.0451, validation error 0.0461
epoch: 11
epoch 11.0000, loss 0.1469, train error 0.0439, validation error 0.0480
epoch: 12
epoc

# Figure out a reasonable Network that achieves good performance
As the last part of this task, setup a network that works well and gets reasonable accuracy, say ~ 1-3 percent error on the **validation set**. 
Train this network on the complete data and compute the **test error**. 

Once you have done this, visualize a few digits from the the test set that the network gets right as well as a few that the network gets wrong!

In [ ]:
# Visualisation here did not work because of some import errors with matplotlib. Added to report instead.

In [13]:
X_test, y_test = Dtest
X_test = X_test.reshape(X_test.shape[0], -1)
test_error = nn.classification_error(X_test, y_test)
print(test_error)

0.032
